# SETTING UP

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC


np.random.seed(1)

# LOAD THE DATA

In [2]:
X_train = pd.read_csv('./airbnb_train_X_price_gte_150.csv') 
y_train = pd.read_csv('./airbnb_train_y_price_gte_150.csv') 
X_test = pd.read_csv('./airbnb_test_X_price_gte_150.csv') 
y_test = pd.read_csv('./airbnb_test_y_price_gte_150.csv') 

In [3]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

# SVM POLYNOMIAL MODEL

In [4]:
svm_poly_model = SVC(kernel="poly", degree=3, coef0=1, C=10)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [7]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with polynomial kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,svm with polynomial kernel,0.867854,0.855839,0.883239,0.869323
0,svm with polynomial kernel,0.867854,0.855839,0.883239,0.869323
0,svm with polynomial kernel,0.867854,0.855839,0.883239,0.869323


# RANDOM SEARCH

In [8]:
score_measure = "precision"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,60),  
    'min_samples_leaf': np.arange(1,60),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 200), 
    'max_depth': np.arange(1,50), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best precision score is 0.8549791238076624
... with parameters: {'min_samples_split': 16, 'min_samples_leaf': 2, 'min_impurity_decrease': 0.0036, 'max_leaf_nodes': 74, 'max_depth': 13, 'criterion': 'entropy'}


C:\Users\18137\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
55 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
55 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\18137\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\18137\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\18137\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (

In [9]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.8481724 Precision=0.8568665 Recall=0.8342750 F1=0.8454198


# GRID SEARCH

In [10]:
score_measure = "precision"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(15,20),  
    'min_samples_leaf': np.arange(1,5),
    'min_impurity_decrease': np.arange(0.0034, 0.0040, 0.0001),
    'max_leaf_nodes': np.arange(70,76), 
    'max_depth': np.arange(10,15), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 4200 candidates, totalling 21000 fits
The best precision score is 0.8605626315693831
... with parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_leaf_nodes': 70, 'min_impurity_decrease': 0.0034, 'min_samples_leaf': 4, 'min_samples_split': 15}


In [11]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.8481724 Precision=0.8568665 Recall=0.8342750 F1=0.8454198


In the Confusion matrix results above, we can see that we have the precision score to be the highest 
compared to the rest of the performance metrics followed by Accuracy and F1 respectively.

Precision=0.8568665
Accuracy=0.8481724
F1=0.8454198
Recall=0.8342750 